In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline
from datetime import datetime

In [ ]:
troca = pd.read_csv('data/events.csv', encoding='UTF-8', low_memory = False, parse_dates = ['timestamp'], na_values=['Unknown'])
######troca['timestamp'] = troca['timestamp'].astype('datetime64[ns]')
troca['marca'] = troca.model.str.split().str.get(0)
troca['dia_semana'] = troca['timestamp'].dt.weekday
# troca_dedup = troca.drop_duplicates()
# troca.count(), troca_dedup.count()
troca.sample()

# Exploración de datos sobre el timestamp


### Sobre troca completo

In [ ]:
troca.set_index('timestamp', inplace=True)
troca

In [ ]:
# eventos ocurridos el 13 de marzo a las 13.50
troca['2018-03-13 13:50']

In [ ]:
#between May 3rd and May 4th
troca['5/3/2018':'5/4/2018']

In [ ]:
troca['5-2018']

In [ ]:
#Count the number of observations per timestamp
troca.groupby(level=0).count()

In [ ]:
# conteo semanal (reshapeo el df para q el indice tome valores semanales)
troca.resample('7d').count().head()

In [ ]:
troca.resample('7D').count().plot()

# Informacion sobre las personas
* varían con el día?
* entran varios días seguidos?
* cambia los fines de semana?

In [ ]:
troca.reset_index(inplace=True)
troca.sample()

In [ ]:
troca_by_user = troca[['person','event', 'dia_semana','timestamp', 'marca']].groupby(['person','event','dia_semana']).count().sample()

In [ ]:
troca[['person','event', 'dia_semana']].groupby(['person','event']).count().plot(kind='bar')


In [ ]:
troca_by_user[(troca_by_user['event'] == 'checkout') and (troca_by_user['event'] != 'conversion')]

## Qué es cada evento:

* “viewed product”: El usuario visita una página de producto.
* “brand listing”: El usuario visita un listado específico de una marca viendo un conjunto de productos.
* “visited site”: El usuario ingresa al sitio a una determinada url.
* “ad campaign hit”: El usuario ingresa al sitio mediante una campana de marketing online.
* “generic listing”: El usuario visita la homepage.
* “searched products”: El usuario realiza una búsqueda de productos en la interfaz de búsqueda del site.
* “search engine hit”: El usuario ingresa al sitio mediante un motor de búsqueda web.
* “checkout”: El usuario ingresa al checkout de compra de un producto.
* “staticpage”: El usuario visita una página
* “conversion”: El usuario realiza una conversión, comprando un producto.
* “lead”: El usuario se registra para recibir una notificación de disponibilidad de stock, para un producto que no se encontraba disponible en ese momento.

In [ ]:
troca['event'].value_counts()

## Viewed product
* vistas por producto
* vistas por producto y pais/ciudad

In [ ]:
view_prod = troca[troca.event == 'viewed product'].dropna(axis=1, how='all').drop('event', axis=1)
view_prod.head(1)

## brand listing 
* vs conversion: cuántos usuarios cambiaron de marca? (entraron buscando una y compraron otra, ver si se puede hacer)
* listados de marca por ciudad/pais

In [ ]:
brand_listing = troca[troca.event == 'brand listing'].dropna(axis=1, how='all').drop('event', axis=1)
brand_listing.head(1)

# Visited site
* visitas al sitio por ciudad
* visitas al sitio por dispositivo/pantalla/SO
* dia semana
* por temporadas


In [ ]:
visited_site = troca[troca.event == 'visited site'].dropna(axis=1, how='all').drop('event', axis=1)
visited_site.head(1)


## Ad campaign hit
Simil anterior
* relacionar con conversiones para sacar si vale la inversión en ads
* ver si se pueden ver números relacionados a la influencia de block ads? (Yep volado esto (?))

In [ ]:
ad_hit = troca[troca.event == 'ad campaign hit'].dropna(axis=1, how='all').drop('event', axis=1)
ad_hit.head(1)

## Generic listing
Similar anterior

In [ ]:
generic_listing = troca[troca.event == 'generic listing'].dropna(axis=1, how='all').drop('event', axis=1)
generic_listing.head(1)

## Searched products

la más interesante del punto de vista de la firma puede ser relacionar SKUS (lo q devuelve una búsqueda) a ciertos terminos de búsqueda y ver si sale algo: si es correcto lo que trae, si trae otras marcas diferentes, si trae cosas que nada que ver, si con alguna búsqueda no trae nada
cuáles son las búsquedas más repeticas y cómo reacciona el buscador (q modelos trae)

>NOTA: HACER UN DF SEPARADO DE MARCAS X SKU. extraer las dos columnas para joinearlas con 'skus' para esto.

In [ ]:
searched_products = troca[troca.event == 'searched products'].dropna(axis=1, how='all').drop('event', axis=1)
searched_products.head(1)

## search engine hit
* fueron más las busquedas por dentro o por fuera.
* hay más de una busqueda por fuera en el mismo tiempo desde el mismo usuario? (AKA puede haber tenido que salir y buscar en google porque nuestro buscador no anda?)?

In [ ]:
search_engine_hit = troca[troca.event == 'search engine hit'].dropna(axis=1, how='all').drop('event', axis=1)
search_engine_hit.head(1)

# checkout

* ver cantidad de checkout por temporada, día de la semana, marca y modelo, SO/pantalla/dispositivo/navegador
* qué caract se venden más: color, storage

> para color estaría bueno hacer otro df 'color primario' y agrupar los colores en categorico (por ej: en rojo: rojo sangre, rojo pasión, colorado, salmon y así).

In [ ]:
checkout = troca[troca.event == 'checkout'].dropna(axis=1, how='all').drop('event', axis=1)
checkout.head(1)

# staticpage
* sacar un unique y ver cuales son
* ver si se puede hacer un analisis del tipo: ¿está la gente buscando mucho los datos de contacto, viendo como comprar?
* Un usuario entró más de una vez a un machete de cómo comprar o a otra? Hay algo ahí?
* se relaciona con la conversion?

In [ ]:
staticpage = troca[troca.event == 'staticpage'].dropna(axis=1, how='all').drop('event', axis=1)
staticpage.head(1)

# conversion
* a qué hora del día la gente compra más.
* lo mismo para día de la semana, pais, pantalla dispositivo navegador
* geolocalizar
* que condicion se vende más (bueno, muy bueno, etc)?

In [ ]:
conversion = troca[troca.event == 'conversion'].dropna(axis=1, how='all').drop('event', axis=1)
conversion.head(1)

In [ ]:
conversion_por_marca = conversion.groupby(['marca', 'model'])
conversion_por_marca[['marca', 'model','condition', 'color']].describe()

In [ ]:
cant_convers_por_marca = conversion_por_marca['timestamp'].count().sort_values(ascending=False)
cant_convers_por_marca.head(10)

In [ ]:
conversion_marca_modelo = conversion.loc[:,['timestamp','marca','model']].groupby(['marca','model']).count().sort_values('timestamp', ascending=False)
conversion_marca_modelo.head()

In [ ]:
cv_marca_modelo = conversion.loc[:,['timestamp','marca','model']].groupby(['marca','model']).count().sort_values('timestamp', ascending=False).head(10).unstack('model').fillna(0)

_ = cv_marca_modelo.plot(kind='bar', figsize=(18,7))
_.set_title('Top 20 de marcas y modelos más vendidos', fontsize=24)
_.set_xlabel('Marca y modelo', fontsize=18)
_.set_ylabel('Cantidad de ventas', fontsize=18)

### Conversion por estado y color

In [ ]:
conversion_por_estetica = conversion.groupby(['condition', 'color'])
conversion_por_estetica.describe()

In [ ]:
conv_caract_esteticas = conversion.loc[:,['timestamp','condition','color']].groupby(['condition','color']).count().sort_values('timestamp', ascending=False)
conv_caract_esteticas.head()

In [ ]:
cv_marca_modelo = conversion.loc[:,['timestamp','condition','color']].groupby(['condition','color']).count().sort_values('timestamp', ascending=False).head(20).unstack('condition').fillna(0)

_ = cv_marca_modelo.plot(kind='bar', figsize=(18,7))
_.set_title('Top 20 de marcas y modelos más vendidos', fontsize=24)
_.set_xlabel('Marca y modelo', fontsize=18)
_.set_ylabel('Cantidad de ventas', fontsize=18)

## Lead
“lead”: El usuario se registra para recibir una notificación de disponibilidad de stock, para un producto que no se encontraba disponible en ese momento.

* hay usuarios que piden cosas muy raras?
* qué modelo esta habitualmente en falta?
* qué caracteristica esta habitualmente en falta? por ejemplo, hay storage de 32 pero piden 64...ver si se puede hacer
* un horario destaca?

In [ ]:
lead = troca[troca.event == 'lead'].dropna(axis=1, how='all').drop('event', axis=1)
lead.sort_values(by='timestamp', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last').head(1)

In [ ]:
lead_by_marca = lead.groupby(['marca', 'model', 'person'])
lead_by_marca[['marca', 'model', 'person','timestamp']].describe()

In [ ]:
# ver el contenido de una determinada marca:
# list(lead_by_marca)

# Ver a quien aplicar una estrategia de carro abandonado (enviar publicidad de un producto que compró):
cant_pedidos_por_marca = lead_by_marca['timestamp'].count().sort_values(ascending=False)
cant_pedidos_por_marca.head(10)

### Join de llenado de países faltantes y grafico de agregación

```python
ev_por_persona = troca.loc[:, ['event', 'person']]
ev_por_pais = (
    ev_por_persona.merge(pais_por_persona, on='person', how='left') # Agregar el país al df
    .groupby(['country', 'event'])['country']
    .count()
    .unstack('event').fillna(0)
)

_ = ev_por_pais.plot(kind='bar', stacked='true', logy=True, figsize=(20,15))
_.set_title('Eventos por país de origen', fontsize=24)
_.set_xlabel('País', fontsize=18)
_.set_ylabel('Cantidad de eventos (escala log.)', fontsize=18)
```


In [ ]:
lead_por_marca_modelo = lead.loc[:,['timestamp','marca','model']].groupby(['marca','model']).count().sort_values('timestamp', ascending=False)
lead_por_marca_modelo.head()

In [ ]:
lead_por_solo_marca = lead.loc[:,['timestamp','marca','model']].groupby(['marca','model']).count().sort_values('timestamp', ascending=False).head(10).unstack('model').fillna(0)

_ = lead_por_solo_marca.plot(kind='bar', figsize=(18,10))
_.set_title('Top 20 de marcas y modelos más pedidos', fontsize=24)
_.set_xlabel('Marca y modelo', fontsize=18)
_.set_ylabel('Cantidad de pedidos', fontsize=18)

### intentos verdes
## No publicar ni sacar conclusiones de esto

In [ ]:
# exploracion de campos por timestamp (carga en un determinado momento, idempotencia y eso)
carga = troca[['event','timestamp']][(troca['event']=='conversion') | (troca['event']=='checkout')].groupby('timestamp').count()
carga.head(1)
ax = sns.heatmap(carga)


In [ ]:
#ax = sns.heatmap(carga)

In [ ]:
lead_by_time = lead[['timestamp']].reset_index().set_index('timestamp').resample('7d').count()#.groupby(['event']).head()
lead_by_time.head()
ax = sns.heatmap(lead_by_time)
ax.set(xlabel='Semana', ylabel='Frecuencia de eventos')
# no publicar esta visu, está MAL usarla para esto, hacer un lineplot o histo

## notas

###### country timezone dataset
https://www.google.com.ar/search?q=countrry+timezone+dataset&oq=countrry+timezone+dataset&aqs=chrome..69i57.8487j0j1&sourceid=chrome&ie=UTF-8